# Generate Cuts to Apply and Mapping to Root Files


In [1]:
import uproot
import numpy as np
import sys
import h5py
import pickle

In [2]:
sys.path.append('../../WatChMaL_analysis_copy/WatChMaL')

from watchmal.dataset.DigiTruthMapping import DigiTruthMapping
from watchmal.dataset.h5_dataset import H5TrueDataset
from watchmal.dataset.cnn_mpmt.cnn_mpmt_dataset import CNNmPMTDataset

## Load Dataset


In [3]:
# Import test events from h5 file
data_path = "/fast_scratch/WatChMaL/data/IWCD_mPMT_Short/IWCD_mPMT_Short_emgp0_E0to1000MeV_digihits.h5"
data_file = h5py.File(data_path, "r")

print(data_file.keys())

angles     = np.array(data_file['angles'])
energies   = np.array(data_file['energies'])
positions  = np.array(data_file['positions'])
labels     = np.array(data_file['labels'])
root_files = np.array(data_file['root_files'])
event_ids  = np.array(data_file['event_ids'])
vetos      = np.array(data_file['veto'])

<KeysViewHDF5 ['angles', 'energies', 'event_hits_index', 'event_ids', 'hit_charge', 'hit_pmt', 'hit_time', 'labels', 'positions', 'root_files', 'veto', 'veto2']>


In [4]:
idxs_path = '/fast_scratch/WatChMaL/data/IWCD_mPMT_Short/index_lists/4class_e_mu_gamma_pi0/IWCD_mPMT_Short_4_class_3M_emgp0_idxs.npz'
idxs = np.load(idxs_path, allow_pickle=True)

train_idxs = idxs['train_idxs']
val_idxs   = idxs['val_idxs']
test_idxs  = idxs['test_idxs']

In [5]:
print(root_files[test_idxs][0:2])

[b'/localscratch/prouse.56905527.0/WCSim/e-/E0to1000MeV/unif-pos-R400-y300cm/4pi-dir/IWCD_mPMT_Short_e-_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir_3000evts_0.root'
 b'/localscratch/prouse.56905527.0/WCSim/e-/E0to1000MeV/unif-pos-R400-y300cm/4pi-dir/IWCD_mPMT_Short_e-_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir_3000evts_0.root']


## Load Run Indices.npy

In [54]:
indices = np.load('/home/zpatel/WatChMaL/outputs/2021-06-15/10-30-10/outputs/indices.npy')

In [56]:
from collections import OrderedDict

In [61]:
print(list(OrderedDict.fromkeys(labels)))
print(list(OrderedDict.fromkeys(labels[test_idxs])))
print(list(OrderedDict.fromkeys(labels[test_idxs][indices])))

[1, 0, 2, 3]
[1, 2, 0, 3]
[1, 0, 2, 3]


## Load vetos

In [6]:
test_vetos  = vetos[test_idxs]
test_labels = labels[test_idxs]

e_OD_veto   = (test_labels == 1) & (test_vetos)
mu_OD_veto  = (test_labels == 2) & (test_vetos)

In [7]:
fitqun_path = '/home/zpatel/WatChMaL_analysis_copy/fitqun_comparison/fitqun_comparison_prep/prep_data/'

## Load True Momenta

In [8]:
momenta = np.load(fitqun_path+'3M_momenta.npz', allow_pickle=True)

test_true_momenta = momenta['test_momenta']

## Load to_wall

In [9]:
to_wall = np.load(fitqun_path+'3M_d_to_wall.npz', allow_pickle=True)

test_to_wall = to_wall['test_d_to_wall'] / 100

## Load d_wall

In [10]:
d_wall = np.load(fitqun_path+'3M_d_wall.npz', allow_pickle=True)

test_d_wall = d_wall['test_d_wall'] / 100

## Load Data Mappings

In [11]:
dtm = DigiTruthMapping(dataset=fitqun_path+'data_for_truth.pkl', mcset=fitqun_path+'truth_for_data.pkl')
print(dtm.get_data_entry(10000))

9825


In [12]:
pion_dtm = DigiTruthMapping(dataset=fitqun_path+'pion_data_for_truth.pkl', mcset=fitqun_path+'pion_truth_for_data.pkl')
print(pion_dtm.get_data_entry(10000))

9572


## Load Root Data

In [13]:
# Retrieve flags
gamma_file_data   = uproot.open('/fast_scratch/WatChMaL/data/IWCD_mPMT_Short/fiTQun/IWCD_mPMT_Short_gamma_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir.fiTQun.root')['fiTQun;1']
e_file_data       = uproot.open('/fast_scratch/WatChMaL/data/IWCD_mPMT_Short/fiTQun/IWCD_mPMT_Short_e-_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir.fiTQun.root')['fiTQun;1']
mu_file_data      = uproot.open('/fast_scratch/WatChMaL/data/IWCD_mPMT_Short/fiTQun/IWCD_mPMT_Short_mu-_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir.fiTQun.root')['fiTQun;1']
pion_file_data    = uproot.open('/fast_scratch/WatChMaL/data/IWCD_mPMT_Short/fiTQun/IWCD_mPMT_Short_pi0_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir.fiTQun.root')['fiTQun;1']

gamma_file_data['fq1rpcflg'].interpretation

AsJagged(AsDtype("('>i4', (7,))"))

## Find indices in true dataset for offset to root files

In [14]:
truehits_filepath = '/data/WatChMaL/data/IWCD_mPMT_Short_emg_E0to1000MeV_truehits.h5'
truehits_dataset = H5TrueDataset(h5_path=truehits_filepath)

In [15]:
print(truehits_dataset.labels.shape)
gamma_base_idx = np.where(truehits_dataset.labels == 0)[0][0]
e_base_idx     = np.where(truehits_dataset.labels == 1)[0][0]
mu_base_idx    = np.where(truehits_dataset.labels == 2)[0][0]
pion_base_idx  = np.where(labels == 3)[0][0]

print(gamma_base_idx)
print(e_base_idx)
print(mu_base_idx)
print(pion_base_idx)

(21000000,)
3000000
0
6000000
20613195


## Construct mapping from test set to fitqun

In [17]:
# truehits_arr = np.zeros_like(test_idxs)
# particle_list = np.zeros_like(test_idxs)

# for i, idx in enumerate(test_idxs):
#     truehits_arr[i] = idx
#     particle_list[i] = labels[idx]
    
    
#     if labels[idx] == 3:
#         truehits_arr[i] = pion_dtm.get_truth_entry(idx % pion_base_idx)

In [18]:
# sorted_truehits = truehits_arr[indices]
# sorted_test_idxs = test_idxs[indices]
# sorted_particle_list = particle_list[indices]

In [19]:
gamma_fq_indices = []
e_fq_indices     = []
mu_fq_indices    = []
pion_fq_indices  = []

for idx in test_idxs:
    particle_label = labels[idx]
    truehits_index = dtm.get_truth_entry(idx)
    
    if particle_label == 0:
        gamma_fq_indices.append(truehits_index % gamma_base_idx)
    elif particle_label == 1:
        e_fq_indices.append(truehits_index)
    elif particle_label == 2:
        mu_fq_indices.append(truehits_index % mu_base_idx)
    elif particle_label == 3:
        # Use pion map instead
        truehits_index = pion_dtm.get_truth_entry(idx % pion_base_idx)
        pion_fq_indices.append(truehits_index)
    
gamma_fq_indices = np.array(gamma_fq_indices)
e_fq_indices     = np.array(e_fq_indices)
mu_fq_indices    = np.array(mu_fq_indices)
pion_fq_indices  = np.array(pion_fq_indices)

In [20]:
# gamma_fq_indices = []
# e_fq_indices     = []
# mu_fq_indices    = []
# pion_fq_indices  = []

# for i, idx in enumerate(sorted_test_idxs):
    
#     if sorted_particle_list[i] == 0:
#         gamma_fq_indices.append(sorted_truehits[i] % gamma_base_idx)
#     elif sorted_particle_list[i] == 1:
#         e_fq_indices.append(sorted_truehits[i])
#     elif sorted_particle_list[i] == 2:
#         mu_fq_indices.append(sorted_truehits[i] % mu_base_idx)
#     elif sorted_particle_list[i] == 3:
#         # Use pion map instead
# #         truehits_index = pion_dtm.get_truth_entry(idx % pion_base_idx)
#         pion_fq_indices.append(sorted_truehits[i])
    
# gamma_fq_indices = np.array(gamma_fq_indices)
# e_fq_indices     = np.array(e_fq_indices)
# mu_fq_indices    = np.array(mu_fq_indices)
# pion_fq_indices  = np.array(pion_fq_indices)

### Load fitqun Flags

In [21]:
gamma_flags = gamma_file_data.arrays('fq1rpcflg')['fq1rpcflg']
e_flags     = e_file_data.arrays('fq1rpcflg')['fq1rpcflg']
mu_flags    = mu_file_data.arrays('fq1rpcflg')['fq1rpcflg']
pion_flags  = pion_file_data.arrays('fq1rpcflg')['fq1rpcflg']

gamma_fq1rpcflg_1 = np.array(gamma_flags[:, 0, 1])
e_fq1rpcflg_1     = np.array(e_flags[:, 0, 1])
mu_fq1rpcflg_1    = np.array(mu_flags[:, 0, 1])
pion_fq1rpcflg_1  = np.array(pion_flags[:, 0, 1])

gamma_fq1rpcflg_2 = np.array(gamma_flags[:, 0, 2])
e_fq1rpcflg_2     = np.array(e_flags[:, 0, 2])
mu_fq1rpcflg_2    = np.array(mu_flags[:, 0, 2])
pion_fq1rpcflg_2  = np.array(pion_flags[:, 0, 2])

In [22]:
test_fq1rpcflg_1 = np.concatenate((e_fq1rpcflg_1[e_fq_indices],
                                  mu_fq1rpcflg_1[mu_fq_indices],
                                  gamma_fq1rpcflg_1[gamma_fq_indices],
                                  pion_fq1rpcflg_1[pion_fq_indices]
                                )) != 0

test_fq1rpcflg_2 = np.concatenate((e_fq1rpcflg_2[e_fq_indices],
                                  mu_fq1rpcflg_2[mu_fq_indices],
                                  gamma_fq1rpcflg_2[gamma_fq_indices],
                                  pion_fq1rpcflg_2[pion_fq_indices]
                                )) != 0


In [23]:
mu_fq1rpcflg_1.shape

(1200000,)

In [24]:
gamma_fqpi0pcflg = np.array(gamma_file_data.arrays('fqpi0pcflg')['fqpi0pcflg'][:, 0])
e_fqpi0pcflg     = np.array(e_file_data.arrays('fqpi0pcflg')['fqpi0pcflg'][:, 0])
mu_fqpi0pcflg    = np.array(mu_file_data.arrays('fqpi0pcflg')['fqpi0pcflg'][:, 0])
pion_fqpi0pcflg  = np.array(pion_file_data.arrays('fqpi0pcflg')['fqpi0pcflg'][:, 0])

In [25]:
test_fqpi0pcflg = np.concatenate((e_fqpi0pcflg[e_fq_indices],
                                mu_fqpi0pcflg[mu_fq_indices],
                                gamma_fqpi0pcflg[gamma_fq_indices],
                                pion_fqpi0pcflg[pion_fq_indices]
                         )) != 0

## Cuts

In [26]:
to_wall_cut = ((test_labels == 0) | (test_labels == 1) | (test_labels == 3)) & (test_to_wall < 0.63*np.log(test_true_momenta) - 2)
d_wall_cut  = test_d_wall < 0.5

In [27]:
fq_comparison = to_wall_cut | d_wall_cut | mu_OD_veto | test_fq1rpcflg_1 | test_fq1rpcflg_2

In [28]:
fq_comparison_pi0 = fq_comparison | test_fqpi0pcflg

In [29]:
fq_comparison_OD_veto = fq_comparison | e_OD_veto

In [69]:
fq_comparison[indices]

array([False, False,  True, ...,  True,  True, False])

In [70]:
cuts = {
        'fq_comparison'        : np.where(fq_comparison)[0],
        'fq_comparison_swap'   : np.where(fq_comparison[indices])[0],
        'fq_comparison_pi0'    : np.where(fq_comparison_pi0)[0],
        'fq_comparison_OD_veto': np.where(fq_comparison_OD_veto)[0],
        'to_wall_cut'          : np.where(to_wall_cut)[0],
        'd_wall_cut'           : np.where(d_wall_cut)[0],
        'e_OD_veto'            : np.where(e_OD_veto3)[0],
        'mu_OD_veto'           : np.where(mu_OD_veto)[0],
        'fq1rpcflg_1'          : np.where(test_fq1rpcflg_1)[0],
        'fq1rpcflg_2'          : np.where(test_fq1rpcflg_2)[0]
        }

In [71]:
fq_comparison_pi0[indices].shape

(4671749,)

In [72]:
fq_comparison_pi0.shape, fq_comparison.shape, fq_comparison_OD_veto.shape, to_wall_cut.shape, d_wall_cut.shape, e_OD_veto.shape, mu_OD_veto.shape, test_fq1rpcflg_1.shape, test_fq1rpcflg_2.shape

((4671749,),
 (4671749,),
 (4671749,),
 (4671749,),
 (4671749,),
 (4671749,),
 (4671749,),
 (4671749,),
 (4671749,))

In [73]:
with open('/home/zpatel/zp_analysis_notebooks/cuts/4_class_3M_fitqun_cuts.pickle', 'wb') as handle:
    pickle.dump(cuts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [74]:
fq_mapping = {
        'gamma_fq_indices': gamma_fq_indices,
        'e_fq_indices'    : e_fq_indices,
        'mu_fq_indices'   : mu_fq_indices,
        'pion_fq_indices' : pion_fq_indices,
        }

In [75]:
print(len(gamma_fq_indices) + len(e_fq_indices) + len(mu_fq_indices) + len(pion_fq_indices))
print(indices.shape[0])

4671749
4671749


In [76]:
with open('/home/zpatel/zp_analysis_notebooks/cuts/4_class_3M_fitqun_mapping.pickle', 'wb') as handle:
    pickle.dump(fq_mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [77]:
fq_mapping

{'gamma_fq_indices': array([      0,       1,       2, ..., 1199997, 1199998, 1199999]),
 'e_fq_indices': array([      0,       1,       2, ..., 1199995, 1199996, 1199999]),
 'mu_fq_indices': array([      0,       1,       4, ..., 1199997, 1199998, 1199999]),
 'pion_fq_indices': array([      0,       1,       2, ..., 1199997, 1199998, 1199999])}

In [ ]:
fq_scores, fq_labels, fq_mom, fq_masses = load_gamma_fq_output(fq_mapping_path, gamma_file_path, e_file_path, mu_file_path, pion_file_path, discriminator='e_v_mu')


short_output_softmax = [short_raw_output_softmax[0][e_gamma_4_class_idxs], short_raw_output_softmax[1], fq_scores[e_gamma_4_class_idxs]]
short_actual_labels  = [short_raw_actual_labels[0][e_gamma_4_class_idxs] , short_raw_actual_labels[1], fq_labels[e_gamma_4_class_idxs]]

'''
short_output_softmax = [short_raw_output_softmax + [fq_scores[e_gamma_4_class_idxs]]
short_actual_labels  = [short_raw_actual_labels  + [fq_labels[e_gamma_4_class_idxs]]
'''